In [0]:
from pyspark.sql.functions import *

In [0]:
df = spark.read \
        .format('json') \
        .option('inferSchema',True) \
        .load('abfss://datamodel@learndatamodelling.dfs.core.windows.net/stocks/raw')

In [0]:
df = df.withColumn('ingestion_date',current_timestamp()) \
       .withColumn('file_path',col('_metadata.file_path'))


In [0]:
df.write \
    .mode('overwrite') \
    .format('delta') \
    .saveAsTable('stocks.bronze_dev.stocks')